## Adiabatic

In [5]:
import os
import time
import glob
import numpy as np
import pandas as pd
from PyLTSpice import SimCommander, RawRead

# =====================================================
# CONFIGURATION
# =====================================================
base_dir = r"D:\LTspice_tuitorial\python\community\Multilayer\Final_results\For_github\Mirror_interlayer\1" 
asc_file = os.path.join(base_dir, "coupled_wein_bridge.asc")

# Separate result folders
fwd_raw_dir = os.path.join(base_dir, "results_raw_forward")
bwd_raw_dir = os.path.join(base_dir, "results_raw_backward")
os.makedirs(fwd_raw_dir, exist_ok=True)
os.makedirs(bwd_raw_dir, exist_ok=True)

param_name = "inter_R"
lambda_forward = np.logspace(np.log10(100), np.log10(5000), 50) #np.arange(1000, 5000, 2000)
lambda_backward = np.logspace(np.log10(5000), np.log10(100), 50) #np.arange(5000, 1000, -2000)
lambda_values = np.concatenate([lambda_forward, lambda_backward])

nodes_to_save = [
    "V(l1vout1)","V(l1vout2)","V(l1vout3)","V(l1vout4)","V(l1vout5)",
    "V(l2vout1)","V(l2vout2)","V(l2vout3)","V(l2vout4)","V(l2vout5)",
    "V(N001)","V(N002)","V(N003)","V(N004)","V(N005)",
    "V(N006)","V(N007)","V(N008)","V(N009)","V(N010)",
    "V(N011)","V(N012)","V(N013)","V(N014)","V(N015)",
    "V(N016)","V(N017)","V(N018)","V(N019)","V(N020)",
    "V(N021)","V(N022)","V(N023)","V(N024)","V(N025)",
    "V(N026)","V(N027)","V(N028)","V(N029)","V(N030)"
]


results = []
final_states = {}

# =====================================================
# MAIN LOOP
# =====================================================
for i, val in enumerate(lambda_values):
    sweep_type = "forward" if i < len(lambda_forward) else "backward"
    print(f"\n=== Step {i+1}/{len(lambda_values)} | {sweep_type.upper()} | {param_name}={val} ===")

    # Create simulation commander
    sim = SimCommander(asc_file)
    sim.set_parameters(**{param_name: val})

    # Inject .ic if available
    if final_states:
        ic_line = ".ic " + " ".join(f"{k}={v:.6f}" for k, v in final_states.items())
        sim.add_instructions(ic_line)

    # Run LTspice simulation
    task = sim.run()

    # =====================================================
    # Locate the latest .raw file (handles _1, _2, etc.)
    # =====================================================
    asc_basename = os.path.splitext(os.path.basename(asc_file))[0]
    pattern = os.path.join(base_dir, asc_basename + "*.raw")

    raw_guess = None
    for _ in range(180):  # wait up to ~3 minutes
        matches = glob.glob(pattern)
        if matches:
            raw_guess = max(matches, key=os.path.getmtime)  # newest file
            break
        time.sleep(1)

    if raw_guess is None or not os.path.exists(raw_guess):
        raise FileNotFoundError(f"No .raw file produced for {param_name}={val}")

    # Move to appropriate directory (forward/backward)
    save_dir = fwd_raw_dir if sweep_type == "forward" else bwd_raw_dir
    raw_path_target = os.path.join(save_dir, f"{param_name}_{val:.0f}.raw")
    os.replace(raw_guess, raw_path_target)
    print(f"→ Saved {raw_path_target}")

    # =====================================================
    # Read final node voltages
    # =====================================================
    try:
        raw = RawRead(raw_path_target)
        available_traces = raw.get_trace_names()

        # Auto-detect if names are "V(...)" or plain
        add_V_prefix = not any(name.startswith("V(") for name in available_traces)

        for node in nodes_to_save:
            node_key = node if node in available_traces else node.replace("V(", "").replace(")", "")
            if add_V_prefix and node_key not in available_traces:
                node_key = f"V({node_key})"
            if node_key not in available_traces:
                continue
            try:
                trace = raw.get_trace(node_key)
                final_states[node] = float(trace.data[-1])
            except Exception:
                pass

    except Exception as e:
        print(f"⚠️ Could not read raw file: {e}")

    results.append({
        "sweep": sweep_type,
        param_name: val,
        **{k: final_states.get(k, np.nan) for k in nodes_to_save}
    })


print("\n✅ All runs completed successfully!")
print(f"✅ Results saved to: {out_file}")
print(f"✅ Forward raw files: {fwd_raw_dir}")
print(f"✅ Backward raw files: {bwd_raw_dir}")



=== Step 1/100 | FORWARD | inter_R=100.0 ===
→ Saved D:\LTspice_tuitorial\python\community\Multilayer\Final_results\For_github\Mirror_interlayer\1\results_raw_forward\inter_R_100.raw

=== Step 2/100 | FORWARD | inter_R=108.3110727436041 ===
→ Saved D:\LTspice_tuitorial\python\community\Multilayer\Final_results\For_github\Mirror_interlayer\1\results_raw_forward\inter_R_108.raw

=== Step 3/100 | FORWARD | inter_R=117.31288478870299 ===
→ Saved D:\LTspice_tuitorial\python\community\Multilayer\Final_results\For_github\Mirror_interlayer\1\results_raw_forward\inter_R_117.raw

=== Step 4/100 | FORWARD | inter_R=127.06284398111256 ===
→ Saved D:\LTspice_tuitorial\python\community\Multilayer\Final_results\For_github\Mirror_interlayer\1\results_raw_forward\inter_R_127.raw

=== Step 5/100 | FORWARD | inter_R=137.623129374475 ===
→ Saved D:\LTspice_tuitorial\python\community\Multilayer\Final_results\For_github\Mirror_interlayer\1\results_raw_forward\inter_R_138.raw

=== Step 6/100 | FORWARD | int


=== Step 43/100 | FORWARD | inter_R=2859.3018398391055 ===
→ Saved D:\LTspice_tuitorial\python\community\Multilayer\Final_results\For_github\Mirror_interlayer\1\results_raw_forward\inter_R_2859.raw

=== Step 44/100 | FORWARD | inter_R=3096.940495707347 ===
→ Saved D:\LTspice_tuitorial\python\community\Multilayer\Final_results\For_github\Mirror_interlayer\1\results_raw_forward\inter_R_3097.raw

=== Step 45/100 | FORWARD | inter_R=3354.329473131718 ===
→ Saved D:\LTspice_tuitorial\python\community\Multilayer\Final_results\For_github\Mirror_interlayer\1\results_raw_forward\inter_R_3354.raw

=== Step 46/100 | FORWARD | inter_R=3633.110235703847 ===
→ Saved D:\LTspice_tuitorial\python\community\Multilayer\Final_results\For_github\Mirror_interlayer\1\results_raw_forward\inter_R_3633.raw

=== Step 47/100 | FORWARD | inter_R=3935.06067024852 ===
→ Saved D:\LTspice_tuitorial\python\community\Multilayer\Final_results\For_github\Mirror_interlayer\1\results_raw_forward\inter_R_3935.raw

=== Step 

→ Saved D:\LTspice_tuitorial\python\community\Multilayer\Final_results\For_github\Mirror_interlayer\1\results_raw_backward\inter_R_359.raw

=== Step 85/100 | BACKWARD | inter_R=331.2019296970979 ===
→ Saved D:\LTspice_tuitorial\python\community\Multilayer\Final_results\For_github\Mirror_interlayer\1\results_raw_backward\inter_R_331.raw

=== Step 86/100 | BACKWARD | inter_R=305.78769216063904 ===
→ Saved D:\LTspice_tuitorial\python\community\Multilayer\Final_results\For_github\Mirror_interlayer\1\results_raw_backward\inter_R_306.raw

=== Step 87/100 | BACKWARD | inter_R=282.32357451071056 ===
→ Saved D:\LTspice_tuitorial\python\community\Multilayer\Final_results\For_github\Mirror_interlayer\1\results_raw_backward\inter_R_282.raw

=== Step 88/100 | BACKWARD | inter_R=260.6599374922931 ===
→ Saved D:\LTspice_tuitorial\python\community\Multilayer\Final_results\For_github\Mirror_interlayer\1\results_raw_backward\inter_R_261.raw

=== Step 89/100 | BACKWARD | inter_R=240.65862417347853 ===
→ 